## Creating Star Schema

In [2]:
import pandas as pd

### Load the dataset

In [4]:
file_path = "combined_hotel_data.csv"
df = pd.read_csv(file_path)

### 1. Create Location Table

In [6]:
df['City'] = df['Location'].str.split(",").str[0].str.strip()
df['Country'] = df['Location'].str.split(",").str[1].str.strip()
location_table = df[['City', 'Country']].drop_duplicates().reset_index(drop=True)
location_table['Location ID'] = range(1, len(location_table) + 1)

##### Merge Location IDs back to main dataset

In [8]:
df = df.merge(location_table, on=['City', 'Country'], how='left')

In [9]:
df.head()

,Location,Hotel Name,Stars,Rating,Review Keyword,Services,Property Offers,City,Country,Location ID
0,"Abu Dhabi, UAE",Copthorne Downtown by Millennium,4.0,8.4,Very good,"['Pool', 'Parking available', 'Spa', 'Breakfas...","{'Room Name': 'Premium Suite', 'Room Charges':...",Abu Dhabi,UAE,1
1,"Abu Dhabi, UAE",The Abu Dhabi Edition,5.0,9.8,Exceptional,"['Hot Tub', 'Pool', 'Bar', 'Spa', 'Room servic...","{'Room Name': 'Deluxe Room, 1 King Bed, Non Sm...",Abu Dhabi,UAE,1
2,"Abu Dhabi, UAE",Southern Sun Abu Dhabi,4.0,9.0,Wonderful,"['Breakfast available', 'Pool', 'Bar', 'Gym', ...","{'Room Name': 'Deluxe Double Room, Non Smoking...",Abu Dhabi,UAE,1
3,"Abu Dhabi, UAE",Khalidiya Palace Rayhaan by Rotana,5.0,9.2,Wonderful,"['Pool', 'On private beach', 'Breakfast availa...","{'Room Name': 'Room, 1 King Bed (Guest)', 'Roo...",Abu Dhabi,UAE,1
4,"Abu Dhabi, UAE",Beach Rotana,5.0,9.0,Wonderful,"['On private beach', 'Pool', 'Breakfast availa...","{'Room Name': 'Room, 1 King Bed, Balcony, Sea ...",Abu Dhabi,UAE,1


### 2. Ensure No Duplicate Hotel Names
##### Count occurrences of Hotel Names

In [11]:
hotel_name_counts = df['Hotel Name'].value_counts()
duplicate_hotels = hotel_name_counts[hotel_name_counts > 1].index.tolist()

##### Remove rows with duplicate Hotel Names

In [13]:
df = df[~df['Hotel Name'].isin(duplicate_hotels)]

### 3. Create Hotel Table

In [15]:
hotel_table = df[['Hotel Name', 'Location ID', 'Stars', 'Rating', 'Review Keyword']].drop_duplicates().reset_index(drop=True)
hotel_table['Hotel ID'] = range(1, len(hotel_table) + 1)

In [16]:
hotel_table = hotel_table[['Hotel ID', 'Hotel Name', 'Location ID', 'Stars', 'Rating', 'Review Keyword']]


##### Merge Hotel IDs back to main dataset

In [18]:
df = df.merge(hotel_table[['Hotel Name', 'Hotel ID']], on='Hotel Name', how='left')

In [19]:
df.head()

,Location,Hotel Name,Stars,Rating,Review Keyword,Services,Property Offers,City,Country,Location ID,Hotel ID
0,"Abu Dhabi, UAE",Southern Sun Abu Dhabi,4.0,9.0,Wonderful,"['Breakfast available', 'Pool', 'Bar', 'Gym', ...","{'Room Name': 'Deluxe Double Room, Non Smoking...",Abu Dhabi,UAE,1,1
1,"Abu Dhabi, UAE",Khalidiya Palace Rayhaan by Rotana,5.0,9.2,Wonderful,"['Pool', 'On private beach', 'Breakfast availa...","{'Room Name': 'Room, 1 King Bed (Guest)', 'Roo...",Abu Dhabi,UAE,1,2
2,"Abu Dhabi, UAE",Beach Rotana,5.0,9.0,Wonderful,"['On private beach', 'Pool', 'Breakfast availa...","{'Room Name': 'Room, 1 King Bed, Balcony, Sea ...",Abu Dhabi,UAE,1,3
3,"Abu Dhabi, UAE",Al Raha Beach Resort and Spa,5.0,9.0,Wonderful,"['Breakfast available', 'On private beach', 'P...",{'Room Name': 'Superior Room King - Free daily...,Abu Dhabi,UAE,1,4
4,"Abu Dhabi, UAE",Grand Villaggio Hotel,3.0,8.0,Very good,"['Pool', 'Spa', 'Gym', 'Laundry', 'Housekeepin...","{'Room Name': 'Deluxe Twin Room, Multiple Beds...",Abu Dhabi,UAE,1,5


### 4. Create Services Table

In [21]:
services_table = df[['Hotel ID', 'Services']].drop_duplicates().reset_index(drop=True)

### 5. Create Property Offers Table (Keep dictionary as-is)

In [23]:
property_offers_table = df[['Hotel ID', 'Property Offers']].drop_duplicates().reset_index(drop=True)

##### Save All Tables to Excel

In [25]:
output_file = "star_schema.xlsx"
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    location_table.to_excel(writer, sheet_name='Location Table', index=False)
    hotel_table.to_excel(writer, sheet_name='Hotel Table', index=False)
    services_table.to_excel(writer, sheet_name='Services Table', index=False)
    property_offers_table.to_excel(writer, sheet_name='Property Offers Table', index=False)
    
print(f"Star schema tables saved to '{output_file}' successfully.")

Star schema tables saved to 'star_schema.xlsx' successfully.
